In [ ]:
import ee
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd
import numpy as np
import time

In [ ]:
ee.Authenticate()
ee.Initialize(project = 'ee-project-id')

Load Wheat Field Coordinate Boundaries

In [ ]:
file_path2 = r"/home/user/Wheat_Field_Polygons/FIELDS_For_Wheat_2020.geojson"
file_path3 = r"/home/user/Wheat_Field_Polygons/FIELDS_For_Wheat_2021.geojson"
file_path4 = r"/home/user/Wheat_Field_Polygons/FIELDS_For_Wheat_2022.geojson"
file_path5 = r"/home/user/Wheat_Field_Polygons/FIELDS_For_Wheat_2023.geojson"
file_path_6 = r"/home/user/Wheat_Field_Polygons/FIELDS_For_Wheat_2024.geojson"
file_paths = [file_path2,file_path3,file_path4,file_path5, file_path_6]

In [ ]:
json_list = []
for file_path in file_paths:
    try:
        with open(file_path, encoding='utf-8', errors='replace') as f:
            json_data = json.load(f)
            json_list.append(json_data)
    except Exception as e:
        print(f'Error processing file {file_path}: {e}')

print(len(json_list))

In [ ]:
fields_2020 = geojson_to_ee(json_list[0])
fields_2021 = geojson_to_ee(json_list[1])
fields_2022 = geojson_to_ee(json_list[2])
fields_2023 = geojson_to_ee(json_list[3])
fields_2024 = geojson_to_ee(json_list[4])

In [ ]:
def create_reduce_region_function(geometry, reducer=ee.Reducer.mean(), scale=10, crs='EPSG:4326', bestEffort=True, maxPixels=1e13, tileScale=4):
    """Creates a function that reduces an Earth Engine image to regional statistics based on given parameters
    
    Returns a function that takes an image as input and outputs an Earth Engine Feature containing the computed statistics"""
    def reduce_region_function(img):
        stat = img.reduceRegion(
            reducer=reducer,
            geometry=geometry, 
            scale=scale,
            crs=crs,
            bestEffort=bestEffort,
            maxPixels=maxPixels,
            tileScale=tileScale)
        return ee.Feature(geometry, stat)
    return reduce_region_function

Downloading NDVI Data for June and July 2020-2024 using Sentinel - 2 with GEE

In [ ]:
def cloud_mask(image):
    """Applies cloud masking to Sentinel-2 images using the Scene Classification Layer (SCL)"""
    scl = image.select('SCL')
    mask = scl.eq(3).Or(scl.gte(7).And(scl.lte(10)))
    return image.updateMask(mask.eq(0))

def calculate_s2_ndvi(image):
    """Calculates NDVI from Sentinel-2 imagery using bands B8 (NIR) and B4 (Red)"""
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

def process_field(feature):
    """Processes Sentinel-2 imagery for a given field feature to extract NDVI statistics
    
    Filters image collection by date and cloud cover, applies cloud masking, 
    calculates NDVI, and returns median NDVI value for the field geometry"""
    field_id = feature.get('Field_ID')
    geometry = feature.geometry()
    s2_50cc = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
               .filterDate(start_date, end_date)
               .filterBounds(geometry)
               .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 50))
               .map(cloud_mask)
               .map(calculate_s2_ndvi)
               .median()
               .select('NDVI')
    )

    def map_func(image):
        stats = image.reduceRegion(
            reducer=ee.Reducer.median().setOutputs(['NDVI']),
            geometry=geometry,
            scale=10,
            maxPixels=1e13,
            bestEffort=True
        )
        ndvi = ee.List([stats.get('NDVI'), -9999]).reduce(ee.Reducer.firstNonNull())
        return ee.Feature(geometry, {
            'Field_ID': field_id,
            'NDVI': ndvi,
            'date': start_date
        })
    
    return map_func(s2_50cc)


In [ ]:
#Batch processes Sentinel-2 satellite imagery to calculate and export NDVI data for agricultural fields across 2020-2024.
date_ranges = [
    ('06-01', '07-01'),
    ('07-01', '08-01')
]

fields = {
    2020: fields_2020,
    2021: fields_2021,
    2022: fields_2022,
    2023: fields_2023,
    2024: fields_2024
}

for year, field in fields.items():
    for start_month, end_month in date_ranges:
        start_date = ee.Date(f'{year}-{start_month}')
        end_date = ee.Date(f'{year}-{end_month}')
        
        all_fields = field.map(process_field)
        all_fields_filtered = all_fields.filter(ee.Filter.neq('NDVI', -9999))
        
        export_task = ee.batch.Export.table.toDrive(
            collection=all_fields_filtered,
            description=f'{start_month}_NEW_NDVI_{year}_export',
            fileFormat='CSV',
            folder='Earth_Engine_Exports'  # This folder will be created in your Google Drive
        )

        export_task.start()
            
        # Wait for the task to complete
        while export_task.active():
            print(f'Waiting for task {export_task.id} to complete...')
            time.sleep(30)  # Wait for 30 seconds before checking again
            
    print(f'Task for {year} completed.')

print("All tasks completed.")

Download Maximum NDVI during the growing season from Sentinel - 2 Images using GEE

In [ ]:
def process_field_max(feature):
    """Processes Sentinel-2 imagery to extract NDVI statistics for each image in collection
    
    Filters imagery by 30% cloud cover threshold, calculates NDVI, and returns collection 
    of features containing field-level NDVI values with corresponding image IDs"""
    field_id = feature.get('Field_ID')
    geometry = feature.geometry()
    s2_50cc = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
               .filterDate(start_date, end_date)
               .filterBounds(geometry)
               .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 30))
               .map(cloud_mask)
               .map(calculate_s2_ndvi)
               .select('NDVI'))

    def map_func(image):
        stats = image.reduceRegion(
            reducer=ee.Reducer.median().setOutputs(['NDVI']),
            geometry=geometry,
            scale=10,
            maxPixels=1e13,
            bestEffort=True
        )
        ndvi = ee.List([stats.get('NDVI'), -9999]).reduce(ee.Reducer.firstNonNull())
        return ee.Feature(geometry, {
            'Field_ID': field_id,
            'NDVI': ndvi,
            'imageID': image.id()
        })
    
    return s2_50cc.map(map_func)



In [ ]:
fields = {
    2020: fields_2020,
    2021: fields_2021,
    2022: fields_2022,
    2023: fields_2023,
    2024: fields_2024
}

for year, fields in fields.items():
    start_date = ee.Date(f'{year}-06-01')
    end_date = ee.Date(f'{year}-08-01')
        
    all_fields = fields.map(process_field_max).flatten()
    all_fields_with_date = all_fields.map(lambda f: f.set('date', ee.String(f.get('imageID')).slice(0, 8)))
    all_fields_filtered = all_fields_with_date.filter(ee.Filter.neq('NDVI', -9999))
        
    export_task = ee.batch.Export.table.toDrive(
        collection=all_fields_filtered,
        description=f'MAX_NDVI_{year}_export',
        fileFormat='CSV',
        folder='Earth_Engine_Exports'  # This folder will be created in your Google Drive
    )

    export_task.start()
        
    # Wait for the task to complete
    while export_task.active():
        print(f'Waiting for task {export_task.id} to complete...')
        time.sleep(30)  # Wait for 30 seconds before checking again
        
    print(f'Task for {year} completed.')

print("All tasks completed.")

Download Climate Data from 

In [ ]:
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
    """Creates a function that reduces an Earth Engine image to regional statistics based on given parameters
Returns a function that takes an image as input and outputs an Earth Engine Feature containing the computed statistics with correct date"""
    def reduce_region_function(img):
        stat = img.reduceRegion(
            reducer=reducer,
            geometry=geometry,
            scale=scale,
            crs=crs,
            bestEffort=bestEffort,
            maxPixels=maxPixels,
            tileScale=tileScale)
        return ee.Feature(geometry, stat).set({'date': img.date().format('YYYY-MM-dd')})
    return reduce_region_function

def process_daily_climate(feature):
    """Extracts ERA5-Land monthly climate variables (temperature, precipitation, radiation, wind) for agricultural fields
    
    Creates a feature collection containing field-level climate statistics using 1km resolution data from ERA5-Land"""
    geometry = feature.geometry()
    field_id = feature.get('Field_ID')
    
    climate_images = (ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")
                  .select("temperature_2m","total_precipitation_sum", "dewpoint_temperature_2m", "surface_solar_radiation_downwards_sum","v_component_of_wind_10m","temperature_2m_min","temperature_2m_max" )
                  .filter(ee.Filter.date(start_date,end_date))
                  )
    
    reduce_climate = create_reduce_region_function(
        geometry=geometry, reducer=ee.Reducer.mean(), scale=1000, crs='EPSG:4326')
    
    climate_stat_fc = ee.FeatureCollection(climate_images.map(reduce_climate))
    
    return climate_stat_fc.map(lambda f: f.set('Field_ID', field_id))


In [ ]:
fields = {
    2020: fields_2020,
    2021: fields_2021,
    2022: fields_2022,
    2023: fields_2023,
    2024: fields_2024
}
years = list(range(2004, 2025))
#all_climate_data = ee.FeatureCollection([])
for year, fields in fields.items():
    start_date = ee.Date(f'{year}-05-01')
    end_date = ee.Date(f'{year}-09-01')
        
    year_climate_data = fields.map(process_daily_climate).flatten()
    
    export_task = ee.batch.Export.table.toDrive(
        collection= year_climate_data,
        description=f'Wheat_Climate_Data_{year}_export',
        fileFormat='CSV',
        folder='Earth_Engine_Exports'  # This folder will be created in your Google Drive
    )

    export_task.start()
        
    # Wait for the task to complete
    while export_task.active():
        print(f'Waiting for task {export_task.id} to complete...')
        time.sleep(30)  # Wait for 30 seconds before checking again
        
    print(f'Task for {year} completed.')

print("All tasks completed.")

Download Soil Info Data using the SOILGRIDS Dataset on GEE

In [ ]:
def process_field(feature):
    geometry = feature.geometry()
    field_id = feature.get('Field_ID')
    
    soil_images = get_soil_images()
    
    soil_data = {}
    for key, img in soil_images.items():
        soil_stats = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=250, 
            maxPixels=1e13,
            bestEffort=True
        )
        soil_data[key] = soil_stats.get(img.bandNames().get(0))
    
    soil_data['Field_ID'] = field_id
    
    return ee.Feature(None, soil_data)

def process_field(feature):
    """Extracts mean soil properties from SoilGrids at 250m resolution for each field
    
    Takes a field feature and returns a new feature containing averaged soil characteristics 
    (bulk density, organic carbon, pH, CEC, texture) with the field ID"""
    geometry = feature.geometry()
    field_id = feature.get('Field_ID')
    
    soil_images = get_soil_images()
    
    soil_data = {}
    for key, img in soil_images.items():
        soil_stats = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=250, 
            maxPixels=1e13,
            bestEffort=True
        )
        soil_data[key] = soil_stats.get(img.bandNames().get(0))
    
    soil_data['Field_ID'] = field_id
    
    return ee.Feature(None, soil_data)

In [ ]:
fields = {
    2020: fields_2020,
    2021: fields_2021,
    2022: fields_2022,
    2023: fields_2023,
    2024: fields_2024
}
#all_climate_data = ee.FeatureCollection([])
for year, fields in fields.items():

    soil_data = fields.map(process_field)
    
    export_task = ee.batch.Export.table.toDrive(
        collection= soil_data,
        description=f'Soil_Data_Flakes{year}_export',
        fileFormat='CSV',
        folder='Earth_Engine_Exports'  # This folder will be created in your Google Drive
    )

    export_task.start()
        
    # Wait for the task to complete
    while export_task.active():
        print(f'Waiting for task {export_task.id} to complete...')
        time.sleep(30)  # Wait for 30 seconds before checking again
        
    print(f'Task for {year} completed.')

print("All tasks completed.")